# Embeddings

## Word2Vec

Векторные модели, которые мы рассматривали до этого, условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations inVector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)

$$\hat{P}(w_1^T) = \prod_{t=1}^T \hat{P}(w_t \mid w_1^{t-1})$$

Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.

### Зачем это нужно?

* Решать лингвистические задачи (в основном это про семантику и сочетаемость)
* Подавать на вход нейронным сетям

### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


<img src="https://nycdatascience.com/blog/wp-content/uploads/2017/06/cossim.png" width="500">


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
1. Матрицы слишком разреженные (→ очень большие)
2. Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстахю Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».

<img src="./img/rusvectores2.png" width="500">

Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же гласса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [53]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer

In [2]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7fcb7efb06a0>)

In [20]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

2019-04-12 15:45:53,489 : INFO : loading projection weights from ruscorpora_mystem_cbow_300_2_2015.bin.gz
2019-04-12 15:46:05,229 : INFO : loaded (281776, 300) matrix from ruscorpora_mystem_cbow_300_2_2015.bin.gz


In [21]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [22]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

2019-04-12 15:46:12,221 : INFO : precomputing L2-norms of word weight vectors


день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S 0.7165195941925049
месяц_S 0.6310489177703857
вечер_S 0.5828738808631897
утро_S 0.5676206946372986
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224717378616333
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946247816085815
утро_S 0.5730193853378296
ноченька_S 0.5582467317581177
рассвет_S 0.5553582906723022
ночка_S 0.5351512432098389
полдень_S 0.5334426760673523
полночь_S 0.478694349527359
день_S 0.4780748784542084
сумерки_S 0.43902185559272766
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  0.04785006]
женщина_S 0.5979775190353394
парень_S 0.499

Находим косинусную близость пары слов:

In [23]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895609343220622


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [24]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [10]:
train = pd.read_csv("./data/w2v/train/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("./data/w2v/train/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv("./data/w2v/train/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" \
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [2]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [50]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set...")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


/home/ancatmara/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Parsing sentences from unlabeled set...


/home/ancatmara/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [51]:
print(len(sentences))
print(sentences[0])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [122]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

2019-04-12 17:21:25,258 : INFO : collecting all words and their counts
2019-04-12 17:21:25,260 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-12 17:21:25,314 : INFO : PROGRESS: at sentence #10000, processed 225800 words, keeping 17774 word types
2019-04-12 17:21:25,368 : INFO : PROGRESS: at sentence #20000, processed 451863 words, keeping 24941 word types
2019-04-12 17:21:25,408 : INFO : PROGRESS: at sentence #30000, processed 671259 words, keeping 30023 word types


Training model...


2019-04-12 17:21:25,465 : INFO : PROGRESS: at sentence #40000, processed 897753 words, keeping 34338 word types
2019-04-12 17:21:25,508 : INFO : PROGRESS: at sentence #50000, processed 1116873 words, keeping 37749 word types
2019-04-12 17:21:25,558 : INFO : PROGRESS: at sentence #60000, processed 1338294 words, keeping 40708 word types
2019-04-12 17:21:25,606 : INFO : PROGRESS: at sentence #70000, processed 1561470 words, keeping 43318 word types
2019-04-12 17:21:25,652 : INFO : PROGRESS: at sentence #80000, processed 1780749 words, keeping 45696 word types
2019-04-12 17:21:25,698 : INFO : PROGRESS: at sentence #90000, processed 2004848 words, keeping 48113 word types
2019-04-12 17:21:25,752 : INFO : PROGRESS: at sentence #100000, processed 2226808 words, keeping 50184 word types
2019-04-12 17:21:25,803 : INFO : PROGRESS: at sentence #110000, processed 2446405 words, keeping 52057 word types
2019-04-12 17:21:25,849 : INFO : PROGRESS: at sentence #120000, processed 2668571 words, keepin

2019-04-12 17:21:29,515 : INFO : PROGRESS: at sentence #760000, processed 16989388 words, keeping 120805 word types
2019-04-12 17:21:29,565 : INFO : PROGRESS: at sentence #770000, processed 17216519 words, keeping 121578 word types
2019-04-12 17:21:29,612 : INFO : PROGRESS: at sentence #780000, processed 17446646 words, keeping 122275 word types
2019-04-12 17:21:29,659 : INFO : PROGRESS: at sentence #790000, processed 17673722 words, keeping 122939 word types
2019-04-12 17:21:29,686 : INFO : collected 123376 word types from a corpus of 17796809 raw words and 795538 sentences
2019-04-12 17:21:29,687 : INFO : Loading a fresh vocabulary
2019-04-12 17:21:29,773 : INFO : min_count=10 retains 34112 unique words (27% of original 123376, drops 89264)
2019-04-12 17:21:29,774 : INFO : min_count=10 leaves 17588810 word corpus (98% of original 17796809, drops 207999)
2019-04-12 17:21:29,888 : INFO : deleting the raw counts dictionary of 123376 items
2019-04-12 17:21:29,891 : INFO : sample=0.001 do

2019-04-12 17:22:24,389 : INFO : EPOCH 3 - PROGRESS: at 48.82% examples, 527831 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:25,393 : INFO : EPOCH 3 - PROGRESS: at 54.27% examples, 541868 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:26,397 : INFO : EPOCH 3 - PROGRESS: at 59.40% examples, 551878 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:27,400 : INFO : EPOCH 3 - PROGRESS: at 64.90% examples, 563022 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:28,415 : INFO : EPOCH 3 - PROGRESS: at 69.37% examples, 564163 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:29,418 : INFO : EPOCH 3 - PROGRESS: at 74.40% examples, 569875 words/s, in_qsize 6, out_qsize 1
2019-04-12 17:22:30,431 : INFO : EPOCH 3 - PROGRESS: at 79.28% examples, 573404 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:31,436 : INFO : EPOCH 3 - PROGRESS: at 84.06% examples, 576002 words/s, in_qsize 7, out_qsize 0
2019-04-12 17:22:32,448 : INFO : EPOCH 3 - PROGRESS: at 88.18% examples, 574143 words/s, in_qsiz

CPU times: user 5min 54s, sys: 1.11 s, total: 5min 55s
Wall time: 1min 50s


Смотрим, сколько в модели слов.

In [123]:
print(len(model_en.wv.vocab))

34112


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [124]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

2019-04-12 17:23:47,048 : INFO : precomputing L2-norms of word weight vectors


[('actress', 0.7272005677223206)]
[('men', 0.6438334584236145)]
[('europe', 0.7301366329193115), ('germany', 0.7224695682525635), ('australia', 0.7180286645889282)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейс модели, чтобы потом сравнить результат с дообученной.

In [132]:
model_en.wv.similarity('lion', 'unicorn')

0.3478375567627835

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [126]:
with open("./data/w2v/train/alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [127]:
model_path = "./data/w2v/movie_reviews/movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

2019-04-12 17:24:04,239 : INFO : saving Word2Vec object under ./data/w2v/movie_reviews/movie_reviews.model, separately None
2019-04-12 17:24:04,241 : INFO : not storing attribute vectors_norm
2019-04-12 17:24:04,242 : INFO : not storing attribute cum_table


Saving model...


2019-04-12 17:24:05,390 : INFO : saved ./data/w2v/movie_reviews/movie_reviews.model


In [128]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

2019-04-12 17:24:18,864 : INFO : loading Word2Vec object from ./data/w2v/movie_reviews/movie_reviews.model
2019-04-12 17:24:19,527 : INFO : loading wv recursively from ./data/w2v/movie_reviews/movie_reviews.model.wv.* with mmap=None
2019-04-12 17:24:19,527 : INFO : setting ignored attribute vectors_norm to None
2019-04-12 17:24:19,528 : INFO : loading vocabulary recursively from ./data/w2v/movie_reviews/movie_reviews.model.vocabulary.* with mmap=None
2019-04-12 17:24:19,528 : INFO : loading trainables recursively from ./data/w2v/movie_reviews/movie_reviews.model.trainables.* with mmap=None
2019-04-12 17:24:19,529 : INFO : setting ignored attribute cum_table to None
2019-04-12 17:24:19,529 : INFO : loaded ./data/w2v/movie_reviews/movie_reviews.model
2019-04-12 17:24:19,615 : INFO : collecting all words and their counts
2019-04-12 17:24:19,616 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-12 17:24:19,623 : INFO : collected 2956 word types from a corpu

(97617, 150225)

Лев и единорог стали гораздо ближе друг к другу!

In [131]:
model.wv.similarity('lion', 'unicorn')

0.48919319531985217

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [134]:
model.init_sims(replace=True)
model_path = "./data/w2v/movie_reviews/movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

2019-04-12 17:27:58,544 : INFO : precomputing L2-norms of word weight vectors
2019-04-12 17:27:58,886 : INFO : storing 34112x300 projection weights into ./data/w2v/movie_reviews/movies_alice.bin


Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния о слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификатор мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [159]:
res = model_ru.accuracy('./data/w2v/evaluation/ru_analogy_tagged.txt')

2019-04-12 17:56:40,962 : INFO : capital-common-countries: 19.0% (58/306)
2019-04-12 17:56:43,182 : INFO : capital-world: 10.1% (52/515)
2019-04-12 17:56:43,727 : INFO : currency: 4.6% (6/130)
2019-04-12 17:56:45,008 : INFO : family: 71.2% (218/306)
2019-04-12 17:56:48,419 : INFO : gram1-Aective-to-adverb: 18.7% (152/812)
2019-04-12 17:56:49,954 : INFO : gram2-opposite: 32.1% (122/380)
2019-04-12 17:56:53,728 : INFO : gram6-nationality-Aective: 32.3% (293/907)
2019-04-12 17:56:53,729 : INFO : total: 26.8% (901/3356)


In [162]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


### Vecto

Также для "внутренней" (intrinsic) оценки, т.е. оценки в отрыве от конкретной задачи, можно использовать фреймворк [vecto](https://vecto.readthedocs.io/en/latest/).

In [31]:
from vecto.benchmarks.analogy import Analogy
from vecto.benchmarks.similarity import Similarity
import vecto.embeddings

2019-04-12 15:54:10,679 : INFO : font search path ['/home/ancatmara/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', '/home/ancatmara/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/afm', '/home/ancatmara/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2019-04-12 15:54:25,215 : INFO : generated new fontManager


Vecto использует свои собственные структуры для работы с векторными моделями, поэтому нужно загрузить модель в память ещё раз. Для простоты загрузим только одну из рассмотренных моделей, Word2Vec.

In [135]:
embeddings = vecto.embeddings.load_from_dir("./data/w2v/movie_reviews/")
embeddings.cache_normalized_copy()

2019-04-12 17:28:10,799 : INFO : Detected VSM in the w2v original binary format


В Vecto можно также как и в Gensim получать вектора для слов или искать наиболее похожие слова из словаря модели:

In [136]:
embeddings.get_most_similar_words('movie')

[['movie', 1.0],
 ['film', 0.9272411],
 ['flick', 0.85202676],
 ['movies', 0.7854172],
 ['it', 0.7843939],
 ['picture', 0.7730175],
 ['sequel', 0.76599324],
 ['documentary', 0.75504],
 ['storyline', 0.7395749],
 ['show', 0.73571754]]

In [144]:
similarity = Similarity()
similarity.get_result(embeddings, path_dataset='./data/w2v/evaluation/similarity')

[{'experiment_setup': {'cnt_found_pairs_total': 2,
   'cnt_pairs_total': 2,
   'embeddings': {'_class': 'vecto.embeddings.legacy_w2v.ModelW2V',
    'normalized': True},
   'category': 'default',
   'dataset': 'ws',
   'method': 'cosine_distance',
   'language': 'en',
   'description': 'TEST FILE',
   'version': '-',
   'measurement': 'spearman',
   'task': 'similarity',
   'timestamp': '2019-04-12T17:46:45.086542',
   'default_measurement': 'spearman'},
  'result': {'spearman': -1},
  'details': []}]

In [143]:
analogy = Analogy()
analogy.get_result(embeddings, path_dataset='./data/w2v/evaluation/analogy')

2019-04-12 17:45:29,938 : INFO : processing ./data/w2v/evaluation/analogy/category1/subcategory_a.txt
N/A% (0 of 2) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--2019-04-12 17:45:30,003 : INFO : processing ./data/w2v/evaluation/analogy/category2/subcategory_b.txt
N/A% (0 of 4) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

[{'details': [{'question verbose': "What is to apple as ['quick'] is to fast",
    'b': 'apple',
    'expected answer': ['banana'],
    'predictions': [{'score': 0.728181004524231,
      'answer': 'lebowski',
      'hit': False},
     {'score': 0.726725697517395, 'answer': 'midget', 'hit': False},
     {'score': 0.7247903347015381, 'answer': 'booty', 'hit': False},
     {'score': 0.7216457724571228, 'answer': 'coat', 'hit': False},
     {'score': 0.7199649214744568, 'answer': 'sauce', 'hit': False},
     {'score': 0.7191541194915771, 'answer': 'horn', 'hit': False}],
    'set_exclude': ['fast', 'apple', 'quick'],
    'rank': 816,
    'landing_b': True,
    'landing_b_prime': False,
    'landing_a': False,
    'landing_a_prime': False,
    'similarity predicted to b_prime cosine': 0.6400811225175858,
    'similarity a to a_prime cosine': 0.7612074613571167,
    'similarity a_prime to b_prime cosine': 0.5334567055106163,
    'similarity b to b_prime cosine': 0.670642763376236,
    'simil

Кроме этих двух задач, существует множество других. Бенчмарки для них можно найти на сайте http://vecto.space.

## Что еще бывает?

## doc2vec

Это word2vec с дополнительной меткой id документа, которая представляет собой отдельный вектор. Вектора id документа и всех слов в нем конкатенируются / усредняются, и таким образом получается вектор документа.

Также реализован в `gensim`: `gensim.models.doc2vec`.


![img](img/w2v_4.png)

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грамов. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

## GloVe

Эмбеддинги от стэнфордской NLP-лаборатории. Похожи на word2vec, но в отличие от него GloVe строит матрицу совместной встречаемости слов и учится на ней, т.е. это гибридный метод. [Вот тут](https://www.quora.com/How-is-GloVe-different-from-word2vec) очень хорошо написано об их отличиях.

* [Статья](https://nlp.stanford.edu/pubs/glove.pdf)
* [Сайт](https://nlp.stanford.edu/projects/glove/) (ссылки на модели прямо на главной)

## BPE

Модель, основанная на кодировании BPE (Byte Pair Encoding). Это один из способов представления текста, в котором мы используем в качестве токена не слова или символы, а наборы символов в зависимости от глубины кодирования. Предобученные BPE-эмбеддинги для разных языков можно свободно скачивать.

* [Статья №1](https://arxiv.org/pdf/1508.07909.pdf) 
* [Статья №2](https://aclweb.org/anthology/L18-1473)
* [Сайт](https://nlp.h-its.org/bpemb/)
* [Репозиторий](https://github.com/bheinzerling/bpemb) библиотеки `bpemb` от авторов второй статьи.

Скажем, мы хотим закодировать aaabdaaabac. В этой последовательности сочетание "aa" встречается наиболее часто. Мы можем "слить" эти буквы вместе и рассматривать их как отдельный символ. И так далее итеративно.  Чем больше размер словаря, тем длиннее будут входящие в него единицы: условно, при объеме словаря = 100 текст разобьется на элементы из 2 символов, а при объеме = 500 — на элементы из 5 символов. Выглядит примерно вот так:

![bpe1](img/bpe1.jpg)

![bpe2](img/bpe2.jpg)

А вот 2D-визуализация для BPE-эмбеддингов для русского.

![bpe](https://nlp.h-its.org/bpemb/ru/ru.wiki.bpe.emb.vs100000.d100.png)


А еще можно посмотреть ее [интерактивную визуализацию](http://projector.tensorflow.org/?config=https://nlp.h-its.org/bpemb/ru/projector.config.json) в TensorFlow Projector.

In [168]:
from bpemb import BPEmb

bpemb_en = BPEmb(lang='en', dim=50)
encoded_text = bpemb_en.encode(' '.join(clean_sents[0]))
print(encoded_text)

2019-04-12 18:32:22,383 : INFO : loading projection weights from /home/ancatmara/.cache/bpemb/en/en.wiki.bpe.vs10000.d50.w2v.bin
2019-04-12 18:32:22,500 : INFO : loaded (10000, 50) matrix from /home/ancatmara/.cache/bpemb/en/en.wiki.bpe.vs10000.d50.w2v.bin


['▁through', '▁the', '▁looking', '-', 'gl', 'ass', '▁by', '▁lewis', '▁car', 'roll', '▁chapter', '▁i', '▁looking', '-', 'gl', 'ass', '▁house', '▁one', '▁thing', '▁was', '▁certain', '▁that', '▁the', '▁white', '▁kit', 'ten', '▁had', '▁had', '▁nothing', '▁to', '▁do', '▁with', '▁it', '▁it', '▁was', '▁the', '▁black', '▁kit', 'ten', '’', 's', '▁fault', '▁entirely']


# Задание

Скачайте [газетный корпус](https://www.dropbox.com/s/7mzom3urqgzyfwa/major_papers.rar?dl=0) (новости из крупных газет на русском языке в xml) или [корпус региональной прессы](https://www.dropbox.com/s/d2gl5atnp7nmgfv/regional_papers.rar?dl=0) (тоже на русском языке, в plain text) и обучите на нем модель word2vec. Можете использовать как оригинальные тексты, так и лемматизированный корпус. 

**NB!** Нужны данные из папок, метаданные в csv-таблицах не понадобятся.

1. Найдите по 5 ближайших слов к словам "город", "деревня", "спорт", "бизнес", "Россия", "происшествие", "река", "озеро", "море", "горы", "депутат", "север", "юг", "кавказ", "сибирь", "газпром". Учтите, что слова может не быть в модели.
2. Посчитайте семантическую близкость слов "театр" и "кино", "Владивосток" и "Москва", "церковь" и "государство", "культура" и "отдых", "преступление" и "наказание".
3. Решите примеры:
        * москва + екатеринбург - собянин
        * спартак - москва + санкт-петербург
        * иркутск - байкал + сочи
        * татарстан - татарский + бурятия
        * чай - лимон + кофе
        * авиакомпания - аэрофлот + ржд
4. Найдите лишнее, попробуйте интерпретировать результаты
        * магазин, супермаркет, рынок, тц
        * теннис, хоккей, футбол, дзюдо
        * кошка, собака, попугай, кролик
        * коми, дагестан, башкирия, камчатка

Сохраните модель, она вам еще понадобится!